# Tugas Besar IF3270 - Pembelajaran Mesin Bag. C

Anggota Kelompok:

1. 13519103 - Bryan Rinaldo
2. 13519135 - Naufal Alexander Suryasumirat
3. 13519141 - Naufal Yahya Kurnianto
4. 13519153 - Maximillian Lukman

In [1]:
import math
import numpy as np

import pandas as pd
from sklearn import preprocessing
from sklearn import datasets
from sklearn.model_selection import train_test_split

## Activation Functions

In [2]:
# Activation functions
## Linear
linear = lambda x: x
linear = np.vectorize(linear)
## Sigmoid
sigmoid = lambda x: 1 / (1 + math.exp(-x))
sigmoid = np.vectorize(sigmoid)
## ReLU
relu = lambda x: float(max(0, x))
relu = np.vectorize(relu)
## Softmax
softmax = lambda x: np.exp(x) / np.exp(x).sum(axis=0) # already used for vectors
## Dict
activation_functions = {
    'linear': linear,
    'sigmoid': sigmoid,
    'relu': relu,
    'softmax': softmax
}

## Loss/Cost Functions

In [3]:
# Loss functions
## Linear, Sigmoid, ReLU
def general_loss(predict: np.array or int, target: list or int):
    if (isinstance(predict, type(int))): return 0.5 * ((target - predict) ** 2)
    sum = 0
    for i in range(len(target)):
        sum += ((target[i] - predict[i]) ** 2)
    return 0.5 * sum
## Softmax
def softmax_loss(predict: np.array, target: int):
    return -math.log(predict[target]) # base e
## Dict
loss_functions = {
    'linear': general_loss,
    'sigmoid': general_loss,
    'relu': general_loss,
    'softmax': softmax_loss
}

## Back-propagation Functions / Derivatives

In [4]:
# Back-propagation functions (derivatives)
## Linear
linear_backprop = lambda x: 1
## Sigmoid
# sigmoid_backprop = lambda x: sigmoid(x) * (1 - sigmoid(x)) # or x * (1 - x)?
sigmoid_backprop = lambda x: x * (1 - x)
## ReLU
relu_backprop = lambda x: float(x >= 0)
relu_backprop = np.vectorize(relu_backprop)
## Softmax
def softmax_backprop(arr, targ):
    arr_copy = np.copy(arr)
    arr_copy[targ] = -(1 - arr_copy[targ])
    return arr_copy
## Dict
backprop_functions = {
    'linear': linear_backprop,
    'sigmoid': sigmoid_backprop,
    'relu': relu_backprop,
    'softmax': softmax_backprop
}

## Layer Class

In [5]:
class Layer:
    # n_neuron: number of neuron, weights: weight matrix, activation: activation function
    def __init__(self, n_neuron: int, weights: np.array, activation: str) -> None:
        self.n_neuron = n_neuron # visualization purposes
        self.weights = weights # weights (including bias)
        self.activation = activation # activation type [linear, sigmoid, relu, softmax]
        self.act_function = activation_functions[activation] # activation function used
        self.loss_function = loss_functions[activation] # loss function used
        self.backprop_functions = backprop_functions[activation] # derivative activation functions
        self.result = [] # retain result of feed forward iteration
        self.deltas = np.zeros_like(self.weights) # initialize delta for backprop
        # every feed forward add result, every backprop add to delta

    def calculate(self, in_matrix: np.array) -> np.array:
        self.result = self.act_function(np.dot(self.weights.transpose(), in_matrix))
        return self.result # [a0, a1, a2, ..., an]

    def calculate_loss(self, prediction: (np.array or int), target: (list or int)) -> float: # used for calculating loss for output layer
        if (self.activation == "softmax"): return self.loss_function(prediction, np.argmax(target))
        return self.loss_function(prediction, target)
    
    def update_weight(self):
        self.weights += self.deltas # adding deltas to weights
        self.deltas = np.zeros_like(self.deltas) # resettings deltas for next mini-batch
        return self.weights # for verbose purpose
    
    def add_deltas(self, delta_matrix: np.array) -> None:
        self.deltas += delta_matrix
        return self.deltas # for verbose purpose
    
    def get_structure(self) -> tuple((int, np.array, np.array)):
        # n_neuron: int, weight matrix: np.array, bias weight matrix: np.array
        n_neuron = self.n_neuron
        weight_neuron = self.weights[:-1,]
        weight_bias = self.weights[-1:,].flatten()
        activation = self.activation
        return (n_neuron, weight_neuron, weight_bias, activation)

## FFNN Class

In [6]:
class FFNN:
    def __init__(self,  
            hidden_layers: list,
            input_layer = None,
            threshold = 0.5,
            learning_rate = 0.01,
            err_threshold = 0.001,
            max_iter = 5000,
            batch_size = 1) -> None:
        self.hidden_layers = hidden_layers
        self.output_layer = hidden_layers[-1]
        self.output_activation = self.output_layer.activation
        self.input_layer = input_layer
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.err_threshold = err_threshold
        self.max_iter = max_iter
        self.batch_size = batch_size # default incremental SGD
    
    @staticmethod
    def generate_model(input_size: int, n_neurons: list, activations: list):
        if (len(n_neurons) != len(activations)): return None
        arr = []
        for i in range(len(n_neurons)):
            if (i == 0): arr.append(Layer(
                n_neurons[i], np.random.uniform(low = -1.0, high = 1.0, 
                    size = (input_size + 1, n_neurons[i])),
                    activations[i])
                )
            else: arr.append(Layer(
                n_neurons[i], np.random.uniform(low = -1.0, high = 1.0,
                    size = (n_neurons[i - 1] + 1, n_neurons[i])),
                    activations[i])
                )
        return FFNN(arr)

    def feed_forward(self) -> (np.array or None):
        if (isinstance(self.input_layer, type(None))): return None
        if len(self.input_layer.shape) == 1: return self.forward(self.input_layer)
        else:
            outputs = []
            for data in self.input_layer: outputs.append(self.forward(data))
            if (self.output_layer.activation == 'softmax'): return outputs
            return np.array(outputs).flatten()
    
    def forward(self, input) -> (np.array or None):
        output = input
        for i in range(0, len(self.hidden_layers)):
            output = self.hidden_layers[i].calculate(np.append(output, 1))
        if (self.output_layer.activation == 'softmax'): return output # usually used for multiclass
        if (self.output_layer.n_neuron > 1): return np.where(output > self.threshold, 1, 0) # multiclass non-softmax
        return int(output > self.threshold) # binary
    
    def fit(self, x_train, y_train, randomize = False, learning_rate = None, 
            batch_size = None, max_iter = None, 
            err_threshold = None, update_every = 250) -> None:
        if learning_rate is not None: self.learning_rate = learning_rate
        if batch_size is not None: self.batch_size = batch_size
        if max_iter is not None: self.max_iter = max_iter
        if err_threshold is not None: self.err_threshold = err_threshold
        for epoch in range(self.max_iter):
            training_data = x_train
            training_target = y_train
            if randomize:
                pass # randomize dataset x_train here
            error_sum = 0 # initialize error (for comparing with err_threshold)
            for iter in range(len(y_train)):
                pred = self.predict(training_data[iter]) # results already encoded
                pred = self.output_layer.result # result before encoded
                error = self.output_layer.calculate_loss(pred, training_target[iter])
                self.backpropagate(training_data[iter], training_target[iter])
                error_sum += error
                if ((iter + 1) % self.batch_size == 0 or iter == len(training_target) - 1):
                    self.update_weights() # update weights (mini-batch)
            err_avg = error_sum / len(y_train)

            if (err_avg < self.err_threshold):
                print("Epoch %d, Loss: %.6f | Reason for stopping: err < err_threshold" % (epoch, err_avg))
                break # stop fitting process when avg error < threshold

            if (epoch % update_every == 0):
                print("Epoch %d, Loss: %.6f" % (1 if epoch == 0 else epoch, err_avg))
        return
    
    def backpropagate(self, input, target): # update deltas for every layer
        err_term = 0
        for iter in reversed(range(0, len(self.hidden_layers))):
            prev_layer = None if iter == 0 else self.hidden_layers[iter - 1]
            prev_result = np.atleast_2d(np.append(input, 1)) if prev_layer == None \
                else np.atleast_2d(np.append(prev_layer.result, 1))
            if (iter == len(self.hidden_layers) - 1): # if output layer
                if (self.output_activation == "softmax"): # if softmax output layer
                    pred = self.output_layer.result
                    err_deriv = self.output_layer.backprop_functions(pred, np.argmax(target))
                    err_term = err_deriv
                    gradient = np.dot(prev_result.T,
                        np.atleast_2d(err_deriv))
                    delta = -self.learning_rate * gradient
                    self.output_layer.add_deltas(delta)
                    pass
                else: # if other output layer
                    pred = self.output_layer.result
                    err_deriv = -(np.array(target) - pred)
                    err_term = err_deriv
                    donet = self.output_layer.backprop_functions(pred)
                    gradient = np.dot(prev_result.T,
                        np.atleast_2d(err_deriv * donet))
                    delta = -self.learning_rate * gradient
                    self.output_layer.add_deltas(delta)
            else: # if hidden layer
                this_layer = self.hidden_layers[iter]
                next_layer = self.hidden_layers[iter + 1]
                err_term = np.add.reduce(next_layer.weights[:-1].T * 
                    np.atleast_2d(err_term).T, 0) / np.shape(err_term)[0]
                donet = this_layer.backprop_functions(this_layer.result) # no softmax in hidden layer
                gradient = np.dot(prev_result.T,
                    np.atleast_2d(err_term * donet))
                delta = -self.learning_rate * gradient
                self.hidden_layers[iter].add_deltas(delta)
                pass
        return
        
    def update_weights(self):
        for layer in self.hidden_layers:
            layer.update_weight()
        return
    
    def attach_input(self, input_layer: np.array) -> None:
        self.input_layer = input_layer
        return

    def attach_hidden_layer(self, hidden_layer: Layer) -> None:
        self.hidden_layers.append(hidden_layer)
        return

    def predict(self, input_layer: np.array) -> list: # input_layer without bias
        self.input_layer = input_layer
        return self.feed_forward()

    def get_structure(self) -> tuple((np.array, list)):
        return (self.input_layer, [layer.get_structure() for layer in self.hidden_layers])
    def save(self) -> None:
      with open("model.txt", "w") as f:
        n_layer_save = len(self.get_structure()[1])
        f.writelines(str(n_layer_save+1))
        f.writelines('\n')

        #initial neuron
        f.writelines(str(len(self.get_structure()[1][0][1])))
        f.writelines('\n')

        for i in range(n_layer_save):
          
          # BIAS
          for bias in self.get_structure()[1][i][2]:
            f.write("%s " % bias)
          f.write('\n')

          # WEIGHT
          countLayer = len(self.get_structure()[1][i][1])
          for j in range(countLayer):
            for weight in self.get_structure()[1][i][1][j]:
              f.write("%s " % weight)
            f.write('\n')

          # ACTIVATION
          f.write(str(self.get_structure()[1][i][3]))
          f.write('\n')

          # NEURON
          f.write(str(self.get_structure()[1][i][0]))
          f.write('\n')
      return

## Accuracy Calculation Function

In [7]:
# Fungsi menghitung akurasi dari model
def calculate_accuracy(model: FFNN, input_set, validation_set: list, is_softmax = False):
    # returns range from 1..100 (percentage)
    predicted_set = model.predict(np.array(input_set))
    if (not isinstance(predicted_set, (list, np.ndarray))): return int(predicted_set == validation_set[0]) * 100
    if (len(predicted_set) != len(validation_set)): return None
    num_correct = 0
    for i in range(len(predicted_set)):
        if is_softmax:
            if (np.argmax(predicted_set[i]) == np.argmax(validation_set[i])): num_correct += 1
        else:
            if predicted_set[i].tolist() == validation_set[i].tolist(): num_correct += 1
    return num_correct / len(validation_set) * 100

## Input Model File Function

In [8]:
# Fungsi membaca input file
def _input(filename: str, with_input = False) -> tuple((FFNN, np.array, np.array)):
    f = open(filename, "r")
    f = f.readlines()
    f = [line.strip() for line in f]

    nLayer = int(f[0])
    f = f[1:]
    n_layer_neurons = []
    struct_model = {}

    for i in range(nLayer-1):
        struct_model[i] = {}
        n_layer_neurons.append(int(f[0]))
        struct_model[i]["b"] = [float(b) for b in f[1].split()]
        struct_model[i]["w"] = [[float(w) for w in weights.split()] for weights in f[2:(2 + int(f[0]))]]
        struct_model[i]["f"] = f[2 + int(f[0])]
        f = f[2 + int(f[0]) + 1:]

    n_layer_neurons.append(int(f[0]))
    
    if (with_input):
        n_input = int(f[1])
        f = f[2:]
        input_data = []
        for i in range(n_input):
            input = [int(x) for x in (f[i].split())]
            input_data.append(input)

        f = f[n_input:]
        validation_data = []
        for i in range(n_input):
            result = [int(y) for y in (f[i].split())]
            validation_data.append(result)

    model_layers = []
    for i in range (nLayer-1):
        weight = struct_model[i]["w"]
        weight.append(struct_model[i]["b"])
        layer = Layer(n_layer_neurons[i+1], np.array(weight), struct_model[i]["f"].lower())
        model_layers.append(layer)
    
    if (with_input):
        return FFNN(model_layers, np.array(input_data)), input_data, validation_data
    else:
        return FFNN(model_layers)

## Show Model Function

In [9]:
# Memperlihatkan koefisien dan struktur dari model
def showModel(model: FFNN): #masukan berupa FFNN
    initLayers = model.get_structure()
    countLayer = len(initLayers[1])
    print("==============Model FFNN==============\n")
    print("------------------------------")
    for j in range(0, countLayer):
        weight = initLayers[1][j][1]
        bias = initLayers[1][j][2]

        if (j == (countLayer - 1)):
            print("------ Output Layer ------" )
            print("Weight: " , weight)
            print("Bias: " , bias)
            print('\n')
            print("------------------------------")
        else:
            print("--- Hidden Layer %d ---" %(j+1))
            print("H%d Weight: " %(j+1), weight)
            print("H%d Bias: " %(j+1), bias)
            print('\n')
            print("------------------------------")

## Fungsi Formatting Input Data

In [10]:
def input_dataset(dataset_input):
  x_iris = dataset_input.data
  y_iris = dataset_input.target

  y_iris_temp = [] 
  maxElement = np.amax(y_iris)
  for x in range(len(y_iris)):
    iris = []
    i = 0

    # CEK UDH FULL
    while len(iris) < (maxElement+1) :
      if(i == y_iris[x]):
        iris.append(1)
      else:
        iris.append(0)
      i+=1
    y_iris_temp.append(iris)

  y_iris_return = np.array(y_iris_temp)
  returnDict = {
      "x_train" : x_iris,
      "y_train" : y_iris_return
  }
  return returnDict

## Fit Kelas FFNN (backprop)

### Load Data

In [11]:
dataIris = datasets.load_iris()
df = pd.DataFrame(data=dataIris.data, columns=dataIris.feature_names)

print("Data Iris")
print(df)

dataset = input_dataset(dataIris) # using input_dataset function

x_train = dataset['x_train']
y_train = dataset['y_train']

Data Iris
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]


## Defining FFNN Model

### Fungsi FFNN.generate_model

Digunakan untuk mendefinisikan model awal yang akan digunakan untuk training (random weight diantara -1.0 dan 1.0)

### Parameters
input_size: size atau panjang dari input data / x_train

n_neurons: banyak neuron untuk tiap layer

activations: fungsi aktivasi untuk setiap layer

In [12]:
ffnn_model = FFNN.generate_model(
    4, [6, 4, 5, 3], ['sigmoid', 'relu', 'linear', 'softmax']
)

## Model Before Training

In [13]:
showModel(ffnn_model) # showing model before training

==============Model FFNN==============

------------------------------
--- Hidden Layer 1 ---
H1 Weight:  [[ 0.34946984  0.27410438  0.15458445 -0.04498611 -0.0901094  -0.14554413]
 [ 0.93971333 -0.26713782  0.04344315  0.64011259 -0.12456725 -0.64132777]
 [ 0.12749986  0.733249   -0.0121222  -0.31632702  0.65377666  0.58979344]
 [ 0.3482548  -0.91214758  0.29261866 -0.84104137 -0.90729138 -0.79656163]]
H1 Bias:  [ 0.92497263  0.74289731  0.53171339 -0.9272321   0.26953848 -0.38956392]


------------------------------
--- Hidden Layer 2 ---
H2 Weight:  [[-0.5622343  -0.22830746 -0.29305007 -0.00358271]
 [ 0.79577188 -0.76972573 -0.95545282 -0.83618171]
 [-0.05578808  0.07422817 -0.06237316  0.52816116]
 [ 0.40612959  0.65568235  0.97817598 -0.02989557]
 [-0.87577106 -0.29758202 -0.97162304  0.2998455 ]
 [ 0.08057758  0.33305817  0.31964299  0.97088272]]
H2 Bias:  [-0.85770576  0.02196974  0.0886889   0.88751857]


------------------------------
--- Hidden Layer 3 ---
H3 Weight:  [[ 3.1

## Training Model

### Parameters
x_train: data untuk training

y_train: target untuk training

learning_rate: learning rate untuk dikalikan dengan gradien

batch_size: size mini-batch

err_threshold: threshold error, kasus berhenti jika error < error threshold

max_iter: iterasi maksimal (epoch)

update_every: memberikan update setiap x iterasi

In [14]:
ffnn_model.fit(x_train, y_train, learning_rate = 0.01, batch_size = 4, err_threshold = 0.01, max_iter = 1000)

Epoch 1, Loss: 1.148503
Epoch 250, Loss: 0.045915
Epoch 500, Loss: 0.049897
Epoch 750, Loss: 0.050231


## Model After Training

In [15]:
showModel(ffnn_model) # showing model after training

==============Model FFNN==============

------------------------------
--- Hidden Layer 1 ---
H1 Weight:  [[ 0.37084848  0.56201544 -0.2931086   0.51863216 -0.23086801 -0.24467107]
 [ 0.9479971  -0.05011123 -0.01965936  0.67227434 -0.21525266 -1.08974467]
 [ 0.13641248  0.72424731  0.29879072 -0.80219975  1.03432402  1.77408187]
 [ 0.34926692 -0.9526872   0.67570248 -1.51158562 -0.61626672 -0.03372059]]
H1 Bias:  [ 0.92853024  0.80470458  0.41154295 -0.77585063  0.21808172 -0.46979416]


------------------------------
--- Hidden Layer 2 ---
H2 Weight:  [[-0.38756216 -0.51373486 -0.52732304 -1.08859591]
 [ 0.92251295 -1.00936113 -1.14894347 -1.81689639]
 [-0.34309958  0.25964261  0.0070495   1.09369377]
 [ 1.6839949  -0.59137883  0.16788834 -3.93095253]
 [-1.20871476 -0.13172877 -0.8946241   0.5914171 ]
 [-1.06721181  1.23125877  0.8051496   3.64097342]]
H2 Bias:  [-0.67729696 -0.26863181 -0.14928566 -0.21499934]


------------------------------
--- Hidden Layer 3 ---
H3 Weight:  [[-4.7

## Predictions After Training

In [16]:
for data in x_train:
    print(ffnn_model.predict(data))

[9.99991802e-01 8.19783757e-06 9.54956400e-20]
[9.99846294e-01 1.53705577e-04 3.71810746e-18]
[9.99959616e-01 4.03844199e-05 7.00055221e-19]
[9.98890520e-01 1.10948013e-03 4.39385443e-17]
[9.99992101e-01 7.89930630e-06 9.11710907e-20]
[9.99984340e-01 1.56603711e-05 2.14368353e-19]
[9.99911922e-01 8.80777236e-05 1.85442498e-18]
[9.99949471e-01 5.05291029e-05 9.26239066e-19]
[9.97668811e-01 2.33118933e-03 1.11126807e-16]
[9.99728760e-01 2.71240269e-04 7.55941165e-18]
[9.99995896e-01 4.10446900e-06 4.02389093e-20]
[9.99654925e-01 3.45074601e-04 1.02122009e-17]
[9.99804278e-01 1.95722286e-04 5.02851037e-18]
[9.99957317e-01 4.26830948e-05 7.50184053e-19]
[9.99999952e-01 4.83733458e-08 1.56754081e-22]
[9.99999787e-01 2.12837963e-07 9.97837819e-22]
[9.99999630e-01 3.69608326e-07 1.98839864e-21]
[9.99991190e-01 8.80981879e-06 1.04483019e-19]
[9.99991891e-01 8.10917070e-06 9.42070374e-20]
[9.99992230e-01 7.77019712e-06 8.93132998e-20]
[9.99883823e-01 1.16177090e-04 2.62086027e-18]
[9.99987475e-

## Accuracy of Model After Training

In [17]:
print("Accuracy of Model: %.2f" % (calculate_accuracy(ffnn_model, x_train, y_train, is_softmax = True)) + "%")

Accuracy of Model: 97.33%


In [18]:
# IRIS MLP

from sklearn.neural_network import MLPClassifier # neural network
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

x, y = load_iris(return_X_y=True, as_frame=True) # First param to denote "dictionary form", second param to declare as pandas df
mlpLearner = MLPClassifier(max_iter = 1000, learning_rate_init = 0.01, batch_size=4, tol=0.001, verbose=False, activation='logistic') # change arguments to customize

# Full Data
mlpLearner.fit(x,y)
predFull = mlpLearner.predict(x)
probFull = mlpLearner.predict_proba(x)
scoreFull = mlpLearner.score(x,y)

print("Class Prediction")
print(predFull)
print("\nProbability Prediction")
print(probFull)
print("\nScore")
print(scoreFull)

Class Prediction
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Probability Prediction
[[9.98982165e-01 1.01783460e-03 8.54158164e-18]
 [9.97913134e-01 2.08686626e-03 5.37779141e-17]
 [9.98510246e-01 1.48975367e-03 2.22033419e-17]
 [9.97516944e-01 2.48305636e-03 9.76303176e-17]
 [9.99056614e-01 9.43385835e-04 7.21360905e-18]
 [9.98603312e-01 1.39668790e-03 3.36178860e-17]
 [9.98175532e-01 1.82446835e-03 5.05552917e-17]
 [9.98621920e-01 1.37808046e-03 2.03251327e-17]
 [9.96884747e-01 3.11525286e-03 1.65107347e-16]
 [9.98363426e-01 1.63657423e-03 2.61510533e-17]
 [9.99162350e-01 8.37650384e-04 5.34358833e-18]
 [9.98245424e-01 1.75457583e-03 4.25668072e-17]
 [9.98331356e-01 1.66864364e-03 2.56963144e-17]
 [9.98692372e-01 1.30762844e-

## Milestone C

## 1. Confusion Matrix Implementation

In [19]:
class ConfusionMatrix:
    def __init__(self, pred, val): # input has to be 2d array [numpy]
        if (len(pred) != len(val)): raise ValueError('pred and val length is not the same!')
        if (len(pred[0]) != len(val[0])): raise ValueError('number of class mismatch!')
        self.pred = pred
        self.val = val
        self.count = len(val) # all data count
        self.num_classes = len(val[0]) # number of classes
        self.num_per_class = [0] * self.num_classes # data count per class

        # col: predicted values, row: actual values [Confusion Matrix]
        self.matrix = self.generate_matrix(pred, val)

        self.true_positives = np.diag(self.matrix, k = 0) # True positives per class
        self.false_positives = self.calculate_false_positives() # False positives per class
        self.false_negatives = self.calculate_false_negatives() # False negatives per class
        self.true_negatives = self.calculate_true_negatives() # True negatives per class

        # Statistics per class
        self.accuracies = np.nan_to_num(self.true_positives / self.num_per_class, copy = True, nan = 1.0) # Accuracy per class
        self.precisions = self.calculate_precisions() # Precision per class
        self.recalls = self.calculate_recalls() # Recall per class
        self.f1_scores = self.calculate_f1_scores() # F1_Score per class

        # Statistics for all
        self.accuracy = np.sum(self.true_positives) / self.count # Accuracy for all
        self.precision = np.average(self.precisions) # Precision for all
        self.recall = np.average(self.recalls) # Recall for all
        self.f1_score = np.average(self.f1_scores) # F1_Score for all

    def generate_matrix(self, pred, val):
        if (len(pred) != len(val)): return None
        matrix = np.zeros(shape=(len(val[0]), len(val[0])))
        for i in range(len(val)):
            matrix[np.argmax(val[i])][np.argmax(pred[i])] += 1
            self.num_per_class[np.argmax(val[i])] += 1
        return matrix
    
    def calculate_false_positives(self):
        arr = []
        for i in range(self.num_classes):
            col = self.matrix[:,i]
            arr.append(np.sum(col) - col[i])
        return np.array(arr)

    def calculate_false_negatives(self):
        arr = []
        for i in range(self.num_classes):
            row = self.matrix[i,:]
            arr.append(np.sum(row) - row[i])
        return np.array(arr)

    def calculate_true_negatives(self):
        arr = []
        for i in range(self.num_classes):
            mat = np.delete(np.delete(self.matrix, i, axis = 1), i, axis = 0)
            arr.append(np.sum(mat))
        return np.array(arr)

    def calculate_precisions(self):
        # true positive / (true positive + false positive)
        return np.nan_to_num(self.true_positives / (self.true_positives + self.false_positives), copy = True, nan = 1.0)

    def calculate_recalls(self):
        # true positive / (true positive + false negative)
        return np.nan_to_num(self.true_positives / (self.true_positives + self.false_negatives), copy = True, nan = 1.0)

    def calculate_f1_scores(self):
        # 2 * (precision * recall) / (precision + recall)
        return np.nan_to_num(2 * (self.precisions * self.recalls) / (self.precisions + self.recalls), copy = True, nan = 1.0)

## 2.1. Hasil Confusion Matrix dan Kinerja Implementasi FFNN

In [20]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Implementasi Confusion Matrix Kelompok
predictTable = []
for data in x_train:
  # predictTable.append(np.argmax(ffnn_model.predict(data)))
  predictStatus = [0 for _ in range(3)]
  predictStatus[np.argmax(ffnn_model.predict(data))] = 1
  predictTable.append(predictStatus)

targetTable = []
for data in y_train:
  # targetTable.append(np.argmax(data))
  targetTable.append(data)

implPredict = np.array(predictTable)
implTarget = np.array(targetTable)
implCM = ConfusionMatrix(implPredict,implTarget)
print(implCM.matrix)
print(implCM.accuracy)
print(implCM.precision)
print(implCM.recall)
print(implCM.f1_score)

# Implementasi Confusion Matrix dan Scoring SKlearn
skPredict = []
skTarget = []
for i in range(len(implPredict)):
  skPredict.append(np.argmax(implPredict[i]))
  skTarget.append(np.argmax(implTarget[i]))

print(confusion_matrix(skTarget, skPredict))
print(accuracy_score(skTarget, skPredict))
print(precision_score(skTarget, skPredict, average='macro'))
print(recall_score(skTarget, skPredict, average='macro'))
print(f1_score(skTarget, skPredict, average='macro'))

[[50.  0.  0.]
 [ 0. 46.  4.]
 [ 0.  0. 50.]]
0.9733333333333334
0.9753086419753086
0.9733333333333333
0.9732905982905983
[[50  0  0]
 [ 0 46  4]
 [ 0  0 50]]
0.9733333333333334
0.9753086419753086
0.9733333333333333
0.9732905982905983


## 2.2. Hasil Confusion Matrix dan Kinerja MLP SKLearn


In [21]:
from sklearn.neural_network import MLPClassifier 
from sklearn.datasets import load_iris
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

x, y = load_iris(return_X_y=True, as_frame=True) 
mlpLearner = MLPClassifier(max_iter = 1000, learning_rate_init = 0.01, batch_size=4, tol=0.001, verbose=False, activation='logistic') 
mlpLearner.fit(x,y)
predFull = mlpLearner.predict(x)
probFull = mlpLearner.predict_proba(x)

# Implementasi Confusion Matrix Kelompok
predictTable = []
for i in range(len(probFull)):
  predictStatus = [0 for _ in range(len(probFull[i]))]
  predictStatus[np.argmax(probFull[i])] = 1
  predictTable.append(predictStatus)

targetTable = []
for data in y_train:
  # targetTable.append(np.argmax(data))
  targetTable.append(data)

mlpPredict = np.array(predictTable)
mlpTarget = np.array(targetTable)


mlpCM = ConfusionMatrix(mlpPredict,mlpTarget)
print(mlpCM.matrix)
print(mlpCM.accuracy)
print(mlpCM.precision)
print(mlpCM.recall)
print(mlpCM.f1_score)

# Implementasi Confusion Matrix dan Scoring SKlearn
print(confusion_matrix(y, predFull))
print(accuracy_score(y, predFull))
print(precision_score(y, predFull, average='macro'))
print(recall_score(y, predFull, average='macro'))
print(f1_score(y, predFull, average='macro'))

[[50.  0.  0.]
 [ 0. 45.  5.]
 [ 0.  0. 50.]]
0.9666666666666667
0.9696969696969697
0.9666666666666667
0.9665831244778613
[[50  0  0]
 [ 0 45  5]
 [ 0  0 50]]
0.9666666666666667
0.9696969696969697
0.9666666666666667
0.9665831244778613


## 3.1. Hasil Confusion Matrix dan Kinerja Implementasi FFNN Split Data

In [22]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

ffnn_model_split = FFNN.generate_model(
    4, [6, 4, 5, 3], ['sigmoid', 'relu', 'linear', 'softmax']
)

implx_train, x_test, imply_train, y_test = train_test_split(x_train, y_train, test_size=0.1,random_state=69)
ffnn_model_split.fit(implx_train, imply_train, learning_rate = 0.01, batch_size = 4, err_threshold = 0.01, max_iter = 1000)

# Implementasi Confusion Matrix Kelompok
predictTable = []
for data in x_test:
  predictStatus = [0 for _ in range(3)]
  predictStatus[np.argmax(ffnn_model_split.predict(data))] = 1
  predictTable.append(predictStatus)

targetTable = []
for data in y_test:
  targetTable.append(data)

implPredict = np.array(predictTable)
implTarget = np.array(targetTable)
implCM = ConfusionMatrix(implPredict,implTarget)
print(implCM.matrix)
print(implCM.accuracy)
print(implCM.precision)
print(implCM.recall)
print(implCM.f1_score)

# Implementasi Confusion Matrix dan Scoring SKlearn
skPredict = []
skTarget = []
for i in range(len(implPredict)):
  skPredict.append(np.argmax(implPredict[i]))
  skTarget.append(np.argmax(implTarget[i]))

print(confusion_matrix(skTarget, skPredict))
print(accuracy_score(skTarget, skPredict))
print(precision_score(skTarget, skPredict, average='macro'))
print(recall_score(skTarget, skPredict, average='macro'))
print(f1_score(skTarget, skPredict, average='macro'))

Epoch 1, Loss: 1.144917
Epoch 250, Loss: 0.085278
Epoch 500, Loss: 0.087789
Epoch 750, Loss: 0.111921
[[6. 0. 0.]
 [0. 3. 0.]
 [0. 0. 6.]]
1.0
1.0
1.0
1.0
[[6 0 0]
 [0 3 0]
 [0 0 6]]
1.0
1.0
1.0
1.0


## 3.2. Hasil Confusion Matrix dan Kinerja MLP SKLearn Split Data


In [23]:
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

x, y = load_iris(return_X_y=True, as_frame=True) 
mlpLearner = MLPClassifier(max_iter = 1000, learning_rate_init = 0.01, batch_size=4, tol=0.001, verbose=False, activation='logistic') 
mlpx_train, mlpx_test, mlpy_train, mlpy_test = train_test_split(x , y, test_size=0.1,random_state=69)
mlpLearner.fit(mlpx_train,mlpy_train)
predSplit = mlpLearner.predict(mlpx_test)
probSplit = mlpLearner.predict_proba(mlpx_test)

# Implementasi Confusion Matrix Kelompok
predictTable = []
for i in range(len(probSplit)):
  predictStatus = [0 for _ in range(len(probSplit[i]))]
  predictStatus[np.argmax(probSplit[i])] = 1
  predictTable.append(predictStatus)

targetTable = []
for data in mlpy_test:
  # targetTable.append(np.argmax(data))
  if (data == 0):
    targetTable.append([1, 0, 0])
  elif (data == 1):
    targetTable.append([0, 1, 0])
  elif (data == 2):
    targetTable.append([0, 0, 1])
  

mlpPredict = np.array(predictTable)
mlpTarget = np.array(targetTable)

mlpCM = ConfusionMatrix(mlpPredict,mlpTarget)
print(mlpCM.matrix)
print(mlpCM.accuracy)
print(mlpCM.precision)
print(mlpCM.recall)
print(mlpCM.f1_score)

# Implementasi Confusion Matrix dan Scoring SKlearn
print(confusion_matrix(mlpy_test, predSplit))
print(accuracy_score(mlpy_test, predSplit))
print(precision_score(mlpy_test, predSplit, average='macro'))
print(recall_score(mlpy_test, predSplit, average='macro'))
print(f1_score(mlpy_test, predSplit, average='macro'))

[[6. 0. 0.]
 [0. 3. 0.]
 [0. 0. 6.]]
1.0
1.0
1.0
1.0
[[6 0 0]
 [0 3 0]
 [0 0 6]]
1.0
1.0
1.0
1.0


## 4. 10 Fold Cross Validation 

In [25]:
import numpy as np
import copy
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)
kf.get_n_splits(x_train)
sum_accuracy = 0
sum_precision = 0
sum_recall = 0
sum_f1_score = 0

ffnn_model_kfold = FFNN.generate_model(
    4, [6, 4, 5, 3], ['sigmoid', 'relu', 'linear', 'softmax']
)

for train_index, test_index in kf.split(x_train):
    x_fold_train, x_fold_test = x_train[train_index], x_train[test_index]
    y_fold_train, y_fold_test = y_train[train_index], y_train[test_index]

    ffnn_train = copy.deepcopy(ffnn_model_kfold)
    ffnn_train.fit(x_fold_train, y_fold_train, learning_rate = 0.01, batch_size = 4, err_threshold = 0.01, max_iter = 1000)
    prediction = []
    target = []
    
    batch_predict = ffnn_train.predict(x_fold_test)

    for predict in batch_predict:
      temp = [0 for _ in range(len(predict))]
      temp[np.argmax(predict)] = 1
      prediction.append(temp)

    cm = ConfusionMatrix(np.array(prediction), np.array(y_fold_test))
    sum_accuracy += cm.accuracy
    sum_precision += cm.precision
    sum_recall += cm.recall
    sum_f1_score += cm.f1_score

print("Kinerja model (accuracy): " + str(sum_accuracy/10))
print("Kinerja model (precision): " + str(sum_precision/10))
print("Kinerja model (recall): " + str(sum_recall/10))
print("Kinerja model (F1 score): " + str(sum_f1_score/10))

Epoch 1, Loss: 1.100823
Epoch 250, Loss: 0.792186
Epoch 500, Loss: 0.051264
Epoch 750, Loss: 0.053507
Epoch 1, Loss: 1.100923
Epoch 250, Loss: 0.849259
Epoch 500, Loss: 0.051266
Epoch 750, Loss: 0.053495
Epoch 1, Loss: 1.100944
Epoch 250, Loss: 0.737263
Epoch 500, Loss: 0.050766
Epoch 750, Loss: 0.053162
Epoch 1, Loss: 1.159035
Epoch 250, Loss: 0.337077
Epoch 500, Loss: 0.049650
Epoch 750, Loss: 0.051080
Epoch 1, Loss: 1.176435
Epoch 250, Loss: 0.374435
Epoch 500, Loss: 0.046729
Epoch 750, Loss: 0.049605
Epoch 1, Loss: 1.176417
Epoch 250, Loss: 0.329547
Epoch 500, Loss: 0.039540
Epoch 750, Loss: 0.034930
Epoch 1, Loss: 1.185059
Epoch 250, Loss: 0.291551
Epoch 500, Loss: 0.048329
Epoch 750, Loss: 0.051185
Epoch 1, Loss: 1.187288
Epoch 250, Loss: 0.268278
Epoch 500, Loss: 0.056926
Epoch 750, Loss: 0.055540
Epoch 1, Loss: 1.187375
Epoch 250, Loss: 0.277662
Epoch 500, Loss: 0.027653
Epoch 750, Loss: 0.025142
Epoch 1, Loss: 1.187288
Epoch 250, Loss: 0.261785
Epoch 500, Loss: 0.046639
Epoch 

## 5. Save Implementation

In [26]:
ffnn_model.save()

## 6. Load Implementation / 7. Prediksi Instance Baru

In [27]:
loaded_model = _input("model.txt", False)

print(loaded_model.predict(x_train[1:4]))

[array([9.99846294e-01, 1.53705577e-04, 3.71810746e-18]), array([9.99959616e-01, 4.03844199e-05, 7.00055221e-19]), array([9.98890520e-01, 1.10948013e-03, 4.39385443e-17])]


## 8. Analisis Hasil Bagian 2 dan 3

Pada bagian 2, kami melakukan penggunaan implementasi kelompok terkait confusion matrix dan penilaian kinerja pada 2 model yang berbeda. Model yang dilakukan penilaian adalah model yang kami bentuk menggunakan implementasi Milestone A dan Milestone B serta implementasi mlp library sklearn. Selain penggunaan implementasi kelompok, juga dilakukan penggunaan confusion matrix dan penilaian kinerja menggunakan library sklearn untuk perbandingan. Seperti yang dapat dilihat pada bagian 2, penggunaan confusion matrix dan penilaian kinerja implementasi kelompok serta penggunaan confusion matrix dan penilaian kinerja sklearn membuahkan hasil yang sama.

Pada bagian 3, hal yang dilakukan sama seperti bagian 2. Namun demikian, terdapat sedikit perbedaan spek terkait data yang digunakan untuk membentuk model dan data yang digunakan untuk memprediksi menggunakan hasil pembelajaran model. Menggunakan split data dengan rasio 90:10, 90% data digunakan untuk melakukan fitting pada model dan 10% data digunakan untuk melakukan prediksi menggunakan model yang telah dilakukan pembelajaran sebelumnya. Sama seperti bagian 2, dapat dilihat pada bagian 3 bahwa penggunaan confusion matrix dan penilaian kinerja implementasi kelompok serta penggunaan confusion matrix dan penilaian kinerja sklearn membuahkan hasil yang sama.

Dari hasil 2 bagian tersebut, dapat diperhatikan bahwa implementasi confusion matrix dan penilaian kinerja yang telah kelompok kami buat membuahkan hasil yang sama dengan menggunakan library sklearn. Maka dari itu, kesimpulan yang kami ambil terkait hal ini adalah bahwa implementasi confusion matrix dan penilaian kinerja yang dibuat oleh kelompok kami sudah benar. 

👍

